# Hidden Markov Model para glosado Otomi

In [1]:
from nltk.tag import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np

In [2]:
file1 = load(open('pickle_objects/predata','rb'))
print(file1[0], len(file1))

[[['n', 'v', 'B-psd'], ['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['μ', 'v', 'I-stem'], ['d', 'v', 'I-stem'], ['i', 'v', 'I-stem']], [['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['ε', 'v', 'I-stem'], ['p', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['í', 'v', 'I-stem']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['t', 'v', 'B-lig'], ["'", 'v', 'B-stem'], ['μ', 'v', 'I-stem'], ['n', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-stem'], ['ä', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['r', 'v', 'I-stem'], ['a', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['á', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['k', 'obl', 'B-stem'], ['o', 'obl', 'I-stem']], [['c', 'obl', 'B-stem'], ['h', 'obl', 'I-stem'], ['í', 'obl', 'I-stem'], ['k', 'obl', 'I-stem'], ['ó', 'obl', 'I-stem'], ['h', 'obl', 'I-stem'], ['t', 'obl', 'I-

In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_set = [list(zip(s,t)) for s,t in train_pairs]

1196 590


### Entrenamiento del modelo

In [5]:
%%time
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_set)

### Evaluación

In [6]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = tagger.tag(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [7]:
Y_true = list(chain(*y_true))
Y_pred = list(chain(*y_pred))
chars,Y_pred = zip(*Y_pred) 
Y_pred = list(Y_pred)

In [8]:
labels = list(set(Y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(Y_true, Y_pred))

Accuracy: 0.8736585052374526


In [9]:
prec, rec, f1, supp = precision_recall_fscore_support(Y_true, Y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
I-que,0.187500,0.750000,0.300000,4.0
I-uno,0.000000,0.000000,0.000000,1.0
B-dual,0.666667,1.000000,0.800000,6.0
B-dem,0.270833,0.881356,0.414343,59.0
B-por.que,0.000000,0.000000,0.000000,1.0
...,...,...,...,...
B-3.pss,0.000000,0.000000,0.000000,15.0
I-pl.exc,0.755814,0.902778,0.822785,72.0
B-coraje,0.000000,0.000000,0.000000,1.0
I-ila,0.683761,0.869565,0.765550,92.0


In [11]:
print(results.to_string())

                  Precision    Recall        F1  Support
I-que              0.187500  0.750000  0.300000      4.0
I-uno              0.000000  0.000000  0.000000      1.0
B-dual             0.666667  1.000000  0.800000      6.0
B-dem              0.270833  0.881356  0.414343     59.0
B-por.que          0.000000  0.000000  0.000000      1.0
I-p.loc            1.000000  0.400000  0.571429      5.0
I-con              0.000000  0.000000  0.000000      2.0
I-1.pls            0.375000  0.750000  0.500000      4.0
B-com              0.333333  1.000000  0.500000      1.0
I-dual             0.666667  1.000000  0.800000      6.0
B-3.pot            0.978022  0.978022  0.978022     91.0
I-2.pot            1.000000  1.000000  1.000000      2.0
I-2.cnt            1.000000  0.714286  0.833333     14.0
I-toluca           1.000000  0.500000  0.666667     24.0
B-3.obj            0.358974  0.823529  0.500000     17.0
B-p.loc            1.000000  0.400000  0.571429      5.0
B-dim              0.784314  0.

In [12]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.8837040664882215
Average recall: 0.8532356897324764
Average F1: 0.8610291091440245
